# 2026 QMW - Genomics Module

January 5, 2026

Author: Erik Owen | Graduate Student in Computational & Systems Biology | Page Lab

# Goals:

- TODO

In [11]:
!curl -L https://raw.githubusercontent.com/erik-owen/2026_QMW_Genomics/main/scripts/bootstrap_colab.sh | bash


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1154  100  1154    0     0   5222      0 --:--:-- --:--:-- --:--:--  5245
> Installing tools...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package tree.
(Reading database ... 117717 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...
Tools installed.

Tool versions:
/usr/bin/bowtie2-align-s version 2.4.4
samtools 1.13
bcftools 1.13
> Downloading workshop data...
  - locus1_hg38_chr11_5215464_5237071_HBB.fa
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                     

In [12]:
!tree

.
├── data
│   └── loci
│       ├── locus1_HBB.ind1_noHbS.reads_R1.fastq.gz
│       ├── locus1_HBB.ind2_HbS_carrier.reads_R1.fastq.gz
│       ├── locus1_hg38_chr11_5215464_5237071_HBB.fa
│       ├── locus1_hg38_chr11_5215464_5237071_HBB.repeats.local.bed
│       ├── locus2_hg38_chr8_96243643_96263643_MTERF3.fa
│       ├── locus2_hg38_chr8_96243643_96263643_MTERF3.repeats.local.bed
│       └── locus2_MTERF3_repeat.ind1_repeat_mix.reads_R1.fastq.gz
└── sample_data
    ├── anscombe.json
    ├── california_housing_test.csv
    ├── california_housing_train.csv
    ├── mnist_test.csv
    ├── mnist_train_small.csv
    └── README.md

3 directories, 13 files
